I will be solving the problem shown using the [Nutils](http://www.nutils.org/en/latest/) Finite Element programming library.

The strong form of the equation is 
$$ - \nabla\cdot\left( D \nabla \phi_0\right) \,+\, \Sigma_a\, \phi_0 = Q_0 $$ where $D = \frac{1}{3\Sigma_t}\,$ and $\,\Sigma_a = \Sigma_t - \Sigma_s^0$

To solve the problem using the Galerkin method, we work with the weak form given by: $$ \int_{\Omega} \nabla N_A \, D\,\nabla \phi_0 + N_A \, \Sigma_a \phi_0 - N_A\, Q_0 \, dx = 0$$ 


In [1]:
from nutils import function, mesh, solver
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
degree = 2 #degree of basis functions
nelems = 10 #number of elements along edge of unit square
W = 1 #width of outer box
Wi = .1 #width of inner box

mat_flag = 'scatterer' #TYPE OF MATERIAL IN OUTER BOX. OTHER POSSIBILITIES: 'reflector' 'absorber' 'air'
if mat_flag == 'scatterer':
    sigt = 2
    sigs = 1.99
elif mat_flag == 'reflector':
    sigt = 2
    sigs = 1.8
elif mat_flag == 'absorber':
    sigt = 10
    sigs = 2
elif mat_flag == 'air':
    sigt = .01
    sigs = .006
    

topo, geom = mesh.unitsquare(nelems, 'square') #unit square centred at (0.5, 0.5)
ns = function.Namespace()
ns.basis = topo.basis('lagrange', degree = degree)

ns.W = W
#ns.Wi = Wi

ns.x = W * geom #scales the unit square to actual square
ns.f = 'max( abs(x_0 - W / 2), abs( x_1 - W / 2) )'   #level set function for inner square 
# function.max( function.abs(ns.x[0] -  W/2), function.abs(ns.x[1] - W/2))
inner, outer = function.partition(ns.f, Wi / 2)

ns.phi = 'basis_A ?dofs_A'
ns.SIGs  = sigs * outer              #scattering cross-section
ns.SIGt  = 0.1 * inner + sigt* outer              #total cross-section
ns.SIGa = 'SIGt - Sigs' #absorption cross-section
ns.D = '1 / (3 SIGt)'   #diffusion co-efficient
ns.Q =  inner  #source term
